# Variable Reflection Experiment Runner

Interactive notebook for running moral reasoning experiments with configurable parameters.

## Setup

In [ ]:
import sys
import os
import re
import asyncio
import pandas as pd
from pathlib import Path
from datetime import datetime

# Add parent directory to path
sys.path.insert(0, '..')
os.chdir('..')

# Load environment variables
with open('.env', 'r') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#'):
            match = re.match(r'(\w+)\s*=\s*["\']?([^"\']+)["\']?', line)
            if match:
                os.environ[match.group(1)] = match.group(2)

import config
print(f"Model: {config.MODEL}")
print(f"Rate limit: {config.CALLS_PER_MINUTE}/min")

## Configuration

Adjust these parameters as needed before running experiments.

In [ ]:
# =============================================================================
# EXPERIMENT PARAMETERS - MODIFY THESE
# =============================================================================

# Reflection levels to test (0-5)
# 0=Direct, 1=Minimal, 2=CoT, 3=Structured, 4=Adversarial, 5=Two-Pass
LEVELS = [0]

# Thinking conditions
# False = Thinking OFF (temperature=0, max_tokens=1000)
# True = Thinking ON (extended thinking with 2000 token budget)
THINKING_CONDITIONS = [False, True]

# Number of runs per condition (for measuring consistency)
N_RUNS = 1

# Sample size per benchmark (None = full dataset)
SAMPLE_SIZE = 6  # Start small for testing

# Include confidence scoring in prompts
INCLUDE_CONFIDENCE = True

# Which benchmarks to run
RUN_ETHICS = True
RUN_MORALCHOICE = True
RUN_MORABLES = True

# Execution mode
USE_ASYNC = True  # True = parallel datasets (~2.5x faster), False = sequential

# =============================================================================
# Summary
# =============================================================================
print("=" * 50)
print("EXPERIMENT CONFIGURATION")
print("=" * 50)
print(f"Levels: {LEVELS}")
print(f"Thinking conditions: {THINKING_CONDITIONS}")
print(f"Runs per condition: {N_RUNS}")
print(f"Sample size: {SAMPLE_SIZE or 'Full dataset'}")
print(f"Confidence scoring: {INCLUDE_CONFIDENCE}")
print(f"Benchmarks: ", end="")
benchmarks = []
if RUN_ETHICS: benchmarks.append("ETHICS")
if RUN_MORALCHOICE: benchmarks.append("MoralChoice")
if RUN_MORABLES: benchmarks.append("MORABLES")
print(", ".join(benchmarks))
print(f"Mode: {'Async (parallel)' if USE_ASYNC else 'Sync (sequential)'}")

# Estimate API calls
calls_per_item = len(LEVELS) - 1 + 2  # Level 5 needs 2 calls
calls_per_item *= len(THINKING_CONDITIONS) * N_RUNS
estimated_items = (SAMPLE_SIZE or 100) * len(benchmarks)
estimated_calls = estimated_items * calls_per_item
print(f"\nEstimated API calls: ~{estimated_calls}")

## Import Experiment Functions

In [ ]:
# Import from run_experiment module
import run_experiment as exp

# Override module-level settings with our config
exp.LEVELS = LEVELS
exp.THINKING_CONDITIONS = THINKING_CONDITIONS
exp.N_RUNS = N_RUNS

# Import API utilities for async mode
from src.api import reset_rate_limiter, get_rate_stats

print("Experiment module loaded and configured.")

## Check Data Files

In [ ]:
# Verify data files exist
data_files = {
    'ETHICS': 'data/ethics_sample.csv',
    'MoralChoice': 'data/moralchoice_sample.csv',
    'MORABLES': 'data/morables/morables_sample.csv'
}

print("Data file status:")
for name, path in data_files.items():
    if Path(path).exists():
        df = pd.read_csv(path)
        print(f"  {name}: {path} ({len(df)} items)")
    else:
        print(f"  {name}: {path} (NOT FOUND - run prepare_data.py)")

## Run Experiments

Choose one of the methods below based on your configuration.

### Option A: Run All Selected Benchmarks

In [ ]:
# Create output directories
Path("results/raw").mkdir(parents=True, exist_ok=True)
Path("results/processed").mkdir(parents=True, exist_ok=True)

start_time = datetime.now()
results = {}

if USE_ASYNC:
    # Async mode - run benchmarks in parallel
    print("Running in ASYNC mode (parallel datasets)...")
    reset_rate_limiter()
    
    async def run_all_async():
        results_queue = asyncio.Queue()
        
        # Start checkpoint writer
        checkpoint_task = asyncio.create_task(exp.checkpoint_writer(results_queue))
        
        tasks = []
        task_names = []
        
        if RUN_ETHICS:
            tasks.append(exp.run_ethics_experiment_async(results_queue, SAMPLE_SIZE, INCLUDE_CONFIDENCE))
            task_names.append('ethics')
        if RUN_MORALCHOICE:
            tasks.append(exp.run_moralchoice_experiment_async(results_queue, SAMPLE_SIZE, INCLUDE_CONFIDENCE))
            task_names.append('moralchoice')
        if RUN_MORABLES:
            tasks.append(exp.run_morables_experiment_async(results_queue, SAMPLE_SIZE, INCLUDE_CONFIDENCE))
            task_names.append('morables')
        
        all_results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Stop checkpoint writer
        checkpoint_task.cancel()
        try:
            await checkpoint_task
        except asyncio.CancelledError:
            pass
        
        return dict(zip(task_names, all_results))
    
    results = await run_all_async()
    
else:
    # Sync mode - run benchmarks sequentially
    print("Running in SYNC mode (sequential)...")
    
    if RUN_ETHICS:
        print("\n" + "="*50 + "\nRUNNING ETHICS\n" + "="*50)
        results['ethics'] = exp.run_ethics_experiment(SAMPLE_SIZE, INCLUDE_CONFIDENCE)
    
    if RUN_MORALCHOICE:
        print("\n" + "="*50 + "\nRUNNING MORALCHOICE\n" + "="*50)
        results['moralchoice'] = exp.run_moralchoice_experiment(SAMPLE_SIZE, INCLUDE_CONFIDENCE)
    
    if RUN_MORABLES:
        print("\n" + "="*50 + "\nRUNNING MORABLES\n" + "="*50)
        results['morables'] = exp.run_morables_experiment(SAMPLE_SIZE, INCLUDE_CONFIDENCE)

end_time = datetime.now()
duration = end_time - start_time

print("\n" + "="*50)
print("EXPERIMENT COMPLETE")
print("="*50)
print(f"Duration: {duration}")

if USE_ASYNC:
    stats = get_rate_stats()
    print(f"Total API calls: {stats['calls']}")
    print(f"Effective rate: {stats['rate']} calls/min")

### Option B: Run Individual Benchmarks

In [ ]:
# Run ETHICS only
ethics_results = exp.run_ethics_experiment(SAMPLE_SIZE, INCLUDE_CONFIDENCE)
print(f"ETHICS: {len(ethics_results)} results")

In [ ]:
# Run MoralChoice only
mc_results = exp.run_moralchoice_experiment(SAMPLE_SIZE, INCLUDE_CONFIDENCE)
print(f"MoralChoice: {len(mc_results)} results")

In [ ]:
# Run MORABLES only
morables_results = exp.run_morables_experiment(SAMPLE_SIZE, INCLUDE_CONFIDENCE)
print(f"MORABLES: {len(morables_results)} results")

## Save Results

In [ ]:
# Save all results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for name, data in results.items():
    if isinstance(data, Exception):
        print(f"{name}: FAILED - {data}")
        continue
    
    if isinstance(data, list):
        df = pd.DataFrame(data)
    else:
        df = data
    
    if df is not None and len(df) > 0:
        # Save to processed folder
        output_path = f"results/processed/{name}_results.csv"
        df.to_csv(output_path, index=False)
        print(f"{name}: Saved {len(df)} results to {output_path}")
        
        # Also save timestamped version
        timestamped_path = f"results/processed/{name}_results_{timestamp}.csv"
        df.to_csv(timestamped_path, index=False)
        print(f"  Also saved to: {timestamped_path}")

## Quick Analysis

In [ ]:
def summarize_results(name, data):
    """Print summary statistics for a benchmark."""
    if isinstance(data, Exception):
        print(f"{name}: FAILED")
        return
    
    if isinstance(data, list):
        df = pd.DataFrame(data)
    else:
        df = data
    
    if df is None or len(df) == 0:
        print(f"{name}: No results")
        return
    
    print(f"\n{'='*50}")
    print(f"{name.upper()} SUMMARY")
    print(f"{'='*50}")
    print(f"Total observations: {len(df)}")
    
    # Extraction success rate
    if 'extracted_answer' in df.columns:
        extraction_rate = df['extracted_answer'].notna().mean()
        print(f"Extraction success rate: {extraction_rate:.1%}")
    
    # Accuracy (for ETHICS and MORABLES)
    if 'correct' in df.columns:
        valid = df[df['correct'].notna()]
        if len(valid) > 0:
            print(f"Overall accuracy: {valid['correct'].mean():.1%}")
            
            # By level
            print("\nAccuracy by level:")
            for level in sorted(df['level'].unique()):
                level_df = valid[valid['level'] == level]
                if len(level_df) > 0:
                    print(f"  Level {level}: {level_df['correct'].mean():.1%}")
            
            # By thinking condition
            print("\nAccuracy by thinking:")
            for thinking in [False, True]:
                think_df = valid[valid['thinking'] == thinking]
                if len(think_df) > 0:
                    label = "ON" if thinking else "OFF"
                    print(f"  Thinking {label}: {think_df['correct'].mean():.1%}")
    
    # Confidence stats
    if 'confidence' in df.columns:
        valid_conf = df[df['confidence'].notna()]
        if len(valid_conf) > 0:
            print(f"\nConfidence stats:")
            print(f"  Mean: {valid_conf['confidence'].mean():.1f}")
            print(f"  Std: {valid_conf['confidence'].std():.1f}")
            print(f"  Range: {valid_conf['confidence'].min():.0f} - {valid_conf['confidence'].max():.0f}")

# Summarize all results
for name, data in results.items():
    summarize_results(name, data)

## Detailed Analysis by Condition

In [ ]:
def create_pivot_table(name, data):
    """Create pivot table of accuracy by level and thinking condition."""
    if isinstance(data, list):
        df = pd.DataFrame(data)
    else:
        df = data
    
    if df is None or 'correct' not in df.columns:
        return None
    
    valid = df[df['correct'].notna()].copy()
    if len(valid) == 0:
        return None
    
    valid['thinking_label'] = valid['thinking'].map({True: 'Thinking ON', False: 'Thinking OFF'})
    
    pivot = valid.pivot_table(
        values='correct',
        index='level',
        columns='thinking_label',
        aggfunc='mean'
    )
    
    return pivot

# Create pivot tables for each benchmark
for name, data in results.items():
    if isinstance(data, Exception) or data is None:
        continue
    
    pivot = create_pivot_table(name, data)
    if pivot is not None:
        print(f"\n{name.upper()} - Accuracy by Level and Thinking")
        display(pivot.style.format("{:.1%}").background_gradient(cmap='RdYlGn', vmin=0, vmax=1))

## View Sample Responses

In [ ]:
def show_sample_responses(name, data, n=3):
    """Show sample responses from results."""
    if isinstance(data, list):
        df = pd.DataFrame(data)
    else:
        df = data
    
    if df is None or len(df) == 0:
        return
    
    print(f"\n{'='*60}")
    print(f"{name.upper()} - Sample Responses")
    print(f"{'='*60}")
    
    sample = df.sample(min(n, len(df)))
    
    for i, (_, row) in enumerate(sample.iterrows()):
        print(f"\n--- Sample {i+1} ---")
        print(f"Level: {row['level']}, Thinking: {'ON' if row['thinking'] else 'OFF'}")
        
        if 'extracted_answer' in row:
            print(f"Extracted answer: {row['extracted_answer']}")
        
        if 'correct' in row:
            print(f"Correct: {row['correct']}")
        
        if 'confidence' in row and pd.notna(row['confidence']):
            print(f"Confidence: {row['confidence']}")
        
        if 'response' in row:
            response = str(row['response'])[:500]
            print(f"\nResponse preview:\n{response}...")

# Show samples from each benchmark
for name, data in results.items():
    if not isinstance(data, Exception) and data is not None:
        show_sample_responses(name, data, n=2)

## Load Previous Results

Use this to load and analyze previously saved results.

In [ ]:
# Load results from files
def load_results():
    """Load all available results files."""
    loaded = {}
    
    files = {
        'ethics': 'results/processed/ethics_results.csv',
        'moralchoice': 'results/processed/moralchoice_results.csv',
        'morables': 'results/processed/morables_results.csv'
    }
    
    for name, path in files.items():
        if Path(path).exists():
            loaded[name] = pd.read_csv(path)
            print(f"Loaded {name}: {len(loaded[name])} results")
        else:
            print(f"{name}: No results file found")
    
    return loaded

# Uncomment to load previous results
# results = load_results()

## Custom Experiment Run

Use this section to run experiments with specific custom parameters.

In [ ]:
# Example: Run just Level 0 and Level 5 to compare direct vs reflective

# Temporarily override settings
exp.LEVELS = [0, 5]  # Only direct and two-pass
exp.THINKING_CONDITIONS = [False]  # Only thinking OFF
exp.N_RUNS = 1

# Run a quick comparison
# comparison_results = exp.run_ethics_experiment(sample_size=20, include_confidence=True)
# print(f"Quick comparison: {len(comparison_results)} results")

# Reset to original settings
exp.LEVELS = LEVELS
exp.THINKING_CONDITIONS = THINKING_CONDITIONS
exp.N_RUNS = N_RUNS

print("Custom experiment cell ready. Uncomment code to run.")